In [161]:
# importing necesssary libraries for anlysis
import pandas as pd
import dask.dataframe as dd
import time

***File loading/reading using pandas and dask***

In [162]:
# loading the dataset using pandas

beginning_time = time.time()
supply_df = pd.read_csv('US_Regional_Sales.csv')
pandas_time = time.time() - beginning_time

In [163]:
# loading the dataset using dask

beginning_time = time.time()
supply_dfdd = dd.read_csv('US_Regional_Sales.csv')
supply_dfdd.head()
dask_time = time.time() - beginning_time

In [164]:
print(f'Pandas took {pandas_time} seconds')
print(f'dask took {dask_time} seconds')

Pandas took 0.06457114219665527 seconds
dask took 0.0691220760345459 seconds


***Data cleaning and validation***

In [165]:
def validate_columns(df):
    df.columns = df.columns.str.replace('_', '').str.strip()
    return df

In [181]:
def removing_space(df):
    df.columns = df.columns.str.replace(' ', '')
    return df

In [182]:
supply_df = validate_columns(supply_df)
supply_df = removing_space(supply_df)

In [171]:
# converting object data type to float and replacing $ with ''

for col in ['UnitCost', 'UnitPrice']:
    supply_df[col] = supply_df[col].str.replace('[$,]', '', regex=True).astype(float)


In [172]:
# Convert OrderDate to datetime
for colm in ['ProcuredDate',	'OrderDate', 'ShipDate', 'DeliveryDate']:
    
    supply_df[colm] = pd.to_datetime(supply_df[colm], format='mixed')


In [173]:
# checking for duplicates

duplicates = supply_df.duplicated(keep= 'last').sum()
print(f'There are {duplicates} duplicates in the columns')

There are 0 duplicates in the columns


***YAML file creation***

In [174]:
supply_df.columns

Index(['OrderNumber', 'SalesChannel', 'WarehouseCode', 'ProcuredDate',
       'OrderDate', 'ShipDate', 'DeliveryDate', 'CurrencyCode', 'SalesTeamID',
       'CustomerID', 'StoreID', 'ProductID', 'OrderQuantity',
       'DiscountApplied', 'UnitCost', 'UnitPrice'],
      dtype='object')

In [178]:
expected_column = ''.join(expected_columns).split(' ')


final = [col.strip()for col in expected_column]

print(final)
print('.....')
print(supply_df.columns)

['OrderNumber', 'Sales', 'Channel', 'WarehouseCode', 'ProcuredDate', 'OrderDate', 'ShipDate', 'DeliveryDate', 'CurrencyCode', 'SalesTeamID', 'CustomerID', 'StoreID', 'ProductID', 'Order', 'Quantity', 'Discount', 'Applied', 'Unit', 'Cost', 'Unit', 'Price']
.....
Index(['OrderNumber', 'SalesChannel', 'WarehouseCode', 'ProcuredDate',
       'OrderDate', 'ShipDate', 'DeliveryDate', 'CurrencyCode', 'SalesTeamID',
       'CustomerID', 'StoreID', 'ProductID', 'OrderQuantity',
       'DiscountApplied', 'UnitCost', 'UnitPrice'],
      dtype='object')


In [185]:
# Validating  columns

import yaml

with open('yaml', 'r') as file:
    schema = yaml.safe_load(file)


expected_columns = (schema['yalm_file']['columns'])
if list(supply_df.columns) != (final):
    raise ValueError('Column names do not match the expected schema')
if len(supply_df.columns) != len(final):
    raise ValueError("Number of columns does not match the expected schema.")

else:
    print('done')

ValueError: Column names do not match the expected schema

In [184]:
print("Expected Columns:", expected_columns)
print('........')
print("DataFrame Columns:", list(supply_df.columns))

Expected Columns: OrderNumber SalesChannel WarehouseCode ProcuredDate OrderDate ShipDate DeliveryDate CurrencyCode SalesTeamID CustomerID StoreID ProductID OrderQuantity DiscountApplied UnitCost UnitPrice
........
DataFrame Columns: ['OrderNumber', 'SalesChannel', 'WarehouseCode', 'ProcuredDate', 'OrderDate', 'ShipDate', 'DeliveryDate', 'CurrencyCode', 'SalesTeamID', 'CustomerID', 'StoreID', 'ProductID', 'OrderQuantity', 'DiscountApplied', 'UnitCost', 'UnitPrice']


***file writting***

In [188]:
final_file = 'cleaned_data.txt.gz'
supply_df.to_csv(final_file, sep='|', index=False, compression='gzip')

***summary***

In [189]:
number_of_rows = len(supply_df)
number_of_columns = len(supply_df.columns)

print(f"The total number of rows: {number_of_rows}")
print(f"The total number of columns: {number_of_columns}")

The total number of rows: 7991
The total number of columns: 16
